In [1]:
import pandas as pd

In [3]:
df_channels = pd.read_csv('/Users/katerynaburovova/PycharmProjects/dehumanization/data/merged_dataset/df_channels.csv', header=None)

FWD from will be used for network analysis

In [4]:
df_channels

,0,1,2,3,4,5,6,7,8,9
0,Unnamed: 0,id,date,from_id,to_id,fwd_from,message,type,duration,channel_name
1,0,2305,2022-11-17 19:08:37+00:00,NaN,PeerChannel(channel_id=1182607945),NaN,Дорогая Маша нарисовала нас с покраденным Енот...,photo,NaN,dolgarevaanna
2,1,2304,2022-11-17 18:56:57+00:00,NaN,PeerChannel(channel_id=1182607945),NaN,"Даже Енот из Херсона в афиге от того, как либе...",text,NaN,dolgarevaanna
3,2,2303,2022-11-17 17:59:12+00:00,NaN,PeerChannel(channel_id=1182607945),NaN,"Хи-хи, лол. А если Стрелков к ним явится? \n- ...",text,NaN,dolgarevaanna
4,3,2302,2022-11-17 14:54:07+00:00,NaN,PeerChannel(channel_id=1182607945),"MessageFwdHeader(date=datetime.datetime(2022, ...","Киров (Вятка! Вятка!), выступаю 3 декабря. \n\...",photo,NaN,dolgarevaanna
...,...,...,...,...,...,...,...,...,...,...
2996,995,139568,2022-11-13 07:24:58+00:00,NaN,PeerChannel(channel_id=1315735637),NaN,‼️‼️‼️Совместного заявления по итогам Восточно...,text,NaN,SolovievLive
2997,996,139567,2022-11-13 07:23:26+00:00,NaN,PeerChannel(channel_id=1315735637),NaN,⚡️Увага! Повітряна тривога оголошена на всій т...,video,11.0,SolovievLive
2998,997,139566,2022-11-13 07:21:21+00:00,NaN,PeerChannel(channel_id=1315735637),NaN,⚡️⚡️⚡️⚡️Демократы получают контроль над Сенато...,text,NaN,SolovievLive
2999,998,139565,2022-11-13 07:20:22+00:00,NaN,PeerChannel(channel_id=1315735637),NaN,NaN,text,NaN,SolovievLive
